In [43]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import json

In [2]:
api_key = "TWfVrlX659GSTS9hcsgUcPZ8uNzfoQsg"

In [ ]:
from ratelimiter import RateLimiter
import requests

# Define the rate limit (e.g., 10 calls per minute)
rate_limiter = RateLimiter(max_calls=10, period=60)

@rate_limiter
def call_mistral_api(data):
    # Replace with your actual Mistral API URL and access token
    url = "https://api.mistral.ai/v1/models/mistral-large:predict"
    headers = {"Authorization": f"Bearer YOUR_ACCESS_TOKEN"}

    # Make the API call with rate limiting
    response = requests.post(url, headers=headers, json=data)

    # Handle the response and potential errors
    if response.status_code == 200:
        return response.json()
    else:
        # Handle error cases (e.g., rate limit exceeded)
        print(f"Error: {response.status_code}")
        return None

# Example usage
data = {"text": "your text here"}

# Call the API with rate limiting
response = call_mistral_api(data)

if response:
    # Process the response from the API
    print(response)

In [13]:
def run_mistral(user_message, model="mistral-large"):
    client = MistralClient(api_key=api_key)
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [28]:
sample_str =  json.dumps({
            "elem_type": "NarrativeText",
            "elem_content": "Section 1. Purpose. Artificial intelligence (AI) holds extraordinary potential for both promise and peril. Responsible AI use has the potential to help solve urgent challenges while making our world more prosperous, productive, innovative, and secure. At the same time, irresponsible use could exacerbate societal harms such as fraud, discrimination, bias, and disinformation; dis- place and disempower workers; stifle competition; and pose risks to national security. Harnessing AI for good and realizing its myriad benefits requires mitigating its substantial risks. This endeavor demands a society-wide effort that includes government, the private sector, academia, and civil society."
        })

In [93]:
def user_message(doc):
    user_message = f"""
    Input:
    Markdown document:
    {doc}

    Instructions:
    Analyze the provided markdown document and identify words and phrases that would be difficult for laymen or everyday people to understand. This includes:
    - Technical jargon: Words specific to a particular field or industry.
    - Archaic language: Words or phrases that are no longer commonly used.
    - Legal terminology: Words or phrases with specific legal meanings.
    - Complex sentence structures: Sentences that are long, convoluted, or difficult to parse.

    Classify the identified words and phrases into three categories based on their difficulty level:
    - Novice: Words or phrases that are unfamiliar to most people.
    - Intermediate: Words or phrases that are somewhat familiar but may require additional context to understand.
    - Expert: Words or phrases that are highly technical or specialized and require significant expertise to understand.

    For each category, create a dictionary where the key is the word/phrase and the value is the corresponding sentence or phrase where it appears in the document.

    Generate a JSON object with the following structure:
    {{
        "text": "[Plain text version of the markdown document]",
        "novice": {{
            "[Word/Phrase 1]": "[Explanation of the Word/Phrase 1 given the text context in which it occurs]",
            ...
        }},
        "intermediate": {{
            "[Word/Phrase 1]": "[Explanation of the Word/Phrase 2 given the text context in which it occurs]",
            ...
        }},
        "expert": {{
            "[Word/Phrase 1]": "[Explanation of the Word/Phrase 3 given the text context in which it occurs]",
            ...
        }}
    }}

    Example:
    Markdown Input:
    # Mr. James

    Mr. James is a 37-year-old, chronically ill individual with a history of respiratory problems. He has been diagnosed with **pulmonary fibrosis**, a progressive lung disease that causes scarring and stiffness in the lungs. This condition makes it difficult for Mr. James to breathe and perform daily activities.

    JSON Output:
    {{
        "text": "Mr. James is a 37-year-old, chronically ill individual with a history of respiratory problems. He has been diagnosed with pulmonary fibrosis, a progressive lung disease that causes scarring and stiffness in the lungs. This condition makes it difficult for Mr. James to breathe and perform daily activities.",
        "novice": {{
            "respiratory problems": "Respiratory problems are conditions that affect breathing, such as asthma and bronchitis.",
            "chronically ill": "A person who has a long-lasting illness or health condition that requires continuous medical care and management.",
            "pulmonary fibrosis": "Pulmonary fibrosis is a lung disease characterized by the scarring and thickening of lung tissue, leading to difficulty in breathing and decreased oxygen levels in the blood.",
            
        }},
        "intermediate": {{
            "chronically ill": "A person who has a long-lasting illness or health condition that requires continuous medical care and management.",
        }},
        "expert": {{
            "pulmonary fibrosis": "Pulmonary fibrosis is a lung disease characterized by the scarring and thickening of lung tissue, leading to difficulty in breathing and decreased oxygen levels in the blood."
        }}
    }}

    Logically, the novice category should contain any phrases that occur in intermediate or expert category. Similarly, the intermediate collection should contain any phrases that occur in expert category.
    Your selection and categorization of words should make logical sense.
    
    Finally, it is highly important to stick to the output format. Here are the expected high-level JSON keys: text, novice, intermediate, expert
    """

    return user_message.strip()

In [68]:
from brave import Brave

brave = Brave(api_key="BSAQLuIuBZoMA5TREUU13PwivwYIBJw")

def call_brave(input):
    num_results = 3
    search_results = brave.search(q=input, count=num_results)
    return search_results.web_results
    



In [114]:

import copy
input_json_files = ["/Users/radhikagaonkar/Documents/dora_mistral_hack/formatted_data_biosafety.json"]


for input_file in input_json_files:
    # Load the JSON file
    with open(input_file, 'r') as f:
        data = json.load(f)

    # Initialize the output dictionary
    all_elems_json=[]

    # Find the "Text" element and extract its content
    count_elems = 0
    
    for element in data['elements']:
        
        if count_elems == 30:
            break

        if element['elem_type'] == 'NarrativeText':
            json_output = {
                'text': "",
            }
            print("Size of narrative text:", len(element['elem_content'].split(" ")))
            
            if len(element['elem_content'].split(" ")) > 50:
                print(element['elem_content'])
                print("   ")
                json_output['text'] = element['elem_content']
                
                imp_words_output = run_mistral(user_message(json.dumps(json_output)))
                print(imp_words_output)
                json_output = json.loads(imp_words_output)
                
                result_url_output = copy.deepcopy(json_output)
                
                for key in json_output:
                    if key in ["novice", "intermediate", "expert"]:
                        words_dict = json_output[key]
                        print("List of extracted words for:", key)
                        print(words_dict.keys())
                        
                        if len(words_dict.keys()) > 0:
                            for word in words_dict:
                                query_str = f"""What would be the meaning of this word or phrase, '{word}' which occurs in the following sentence '{words_dict[word]}'"""
                                print("Query:")
                                print(query_str)
                                print(" ")
                                try:
                                    search_result = call_brave(query_str)
                                    print("Search result:")
                                    print(search_result)
                                    print("Number of search results:", len(search_result))
                                    print(search_result[0])
                                    print(search_result[0].keys())
                                    print(search_result[0]["url"])
                                    print(search_result[0]["description"])
                                    result_url_output[key][word] = {"explanation": json_output[key][word], "url": str(search_result[0]["url"]), "title": search_result[0]["title"]}
                                    print("   ")

                                except Exception as e:
                                    print(e)
                                    result_url_output[key][word] = {"explanation": json_output[key][word], "url": "", "title": ""}
                                    
            else:
                result_url_output = {"text": element['elem_content'], "novice": {}, "intermediate": {}, "expert": {}}                    

                    
            print("   ")
            
            print("-------------------------------------------------------------------------")
            
            print("    ")
            
            all_elems_json.append(result_url_output)
            count_elems += 1

    

Size of narrative text: 160
FOREWORD It is a vital interest of the United States to manage the risk of biological incidents, whether naturally occurring, accidental, or deliberate. This National Biodefense Strategy and Implementation Plan for Countering Biological Threats, Enhancing Pandemic Preparedness, and Achieving Global Health Security updates the 2018 National Biodefense Strategy and serves as a foundational component of the President’s vision to create a world free from catastrophic biological incidents, laying out a set of objectives to effectively counter the spectrum of biological threats. It defines biodefense as actions to counter biological threats, reduce biological risks, and prepare for, respond to, and recover from biological incidents, whether naturally occurring, accidental, or deliberate in origin and whether impacting human, animal, plant, or environmental health. It is broader than a Federal Government strategy; it is a call to action for state, local, tribal, an

In [115]:

#write a jsonl output
output = "30_biosafety_output_url_samples_v2.jsonl"

with open(output, 'w') as f:
    for result in all_elems_json:
            json_str = json.dumps(result)  # Convert object to JSON string
            f.write(json_str + '\n')
